In [0]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from itertools import count
from IPython.display import clear_output
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from scipy.stats import pearsonr
from tqdm import tqdm

In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
MODEL_ADDITION_DELTA = 0.01
MODEL_ADDITION_PATIENCE = 3
NR_OF_RUNS = 10
MODEL_NAME = "MNIST_bagging"
PATH = ""

# Preprocess

In [0]:
def preprocess(imgs):
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [0]:
(x_train_val, y_train_val), (x_test, y_test) = mnist.load_data()

x_train_val = preprocess(x_train_val)
x_test = preprocess(x_test)

print('x_train shape:', x_train_val.shape)
print(x_train_val.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [0]:
# Convert class vectors to binary class matrices.
y_train_val = keras.utils.to_categorical(y_train_val, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train_val = x_train_val.astype('float32')
x_test = x_test.astype('float32')
x_train_val /= 255
x_test /= 255

# Model

In [0]:
def MNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size = (2,2), strides=(2,2), padding = "same")(x)
    x = Conv2D(filters=32, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters = 10, kernel_size = (1,1),strides = (1,1), padding = 'valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

# Predict

In [0]:
def predict(models, X, Y):
    predictions = np.empty((len(X),0,NUM_CLASSES))

    for m in tqdm(models):
        pred = np.expand_dims(m.predict(X), axis=1)
        predictions = np.append(predictions, pred, axis=1)

    predictions = np.apply_along_axis(np.transpose, axis=1, arr=predictions)
    predictions = np.mean(predictions, axis=1)
    prediction = np.argmax(predictions, axis=1)

    return accuracy_score(prediction, np.argmax(Y, axis=1))

# Train

In [0]:
for run in range(0, NR_OF_RUNS):

    # Set the seeds
    np.random.seed(run)
    tf.random.set_seed(run)

    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.20, shuffle= True)

    models = []
    accuracies = [0]
    patience = 0

    for i in count(1):

        print(f"Train model {i}")

        # Create directories
        os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
        os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)

        # Bootstrapping
        idx = np.random.choice(len(x_train), size=len(x_train), replace=True)

        x_train_model = x_train[idx]
        y_train_model = y_train[idx]

        # Create the model
        model = MNISTmodel(IMAGE_SIZE, NUM_CLASSES, 1)
        
        es = EarlyStopping(min_delta=0.01, patience=3)
        csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history-{i}.csv", separator=';')

        model.fit(x_train_model,y_train_model,
                  batch_size = BATCH_SIZE,
                  epochs = EPOCHS,
                  validation_data = (x_val, y_val),
                  shuffle = True,
                  callbacks=[es, csv_logger])
        
        model.save_weights(PATH + MODEL_NAME + f"/{run}/weights/weights-{i}.h5" )
        models.append(model)

        acc = predict(models, x_val, y_val)
        delta = acc - accuracies[-1]

        accuracies.append(acc)

        if delta >= MODEL_ADDITION_DELTA:
          patience = 0
        else:
          patience += 1

        print(f"Model: {i} added. Resulting score: {acc}, Delta: {delta}, Patience: {patience}")

        if patience >= MODEL_ADDITION_PATIENCE:
          break

    # Results

    ## Accuracy vs nr of models
    ## Visualizing the accuracy vs the number of models in the ensamble

    print("Accuracy vs nr of models")

    accuracy_df = pd.DataFrame(accuracies, columns=["Accuracy"])
    accuracy_df.insert(1, "Nr of models", accuracy_df.index)

    display(accuracy_df)
    accuracy_df.to_csv(PATH + MODEL_NAME + f"/{run}/accuracy.csv")


    accuracy_df = accuracy_df.iloc[1:]
    accuracy_df.plot(x="Nr of models", y="Accuracy", xticks=accuracy_df["Nr of models"])
    plt.show()

    ## Accuracy
    ## The final accuracy of the ensamble on the test set
    print("Accuracy")

    accuracy = predict(models, x_test, y_testc)
    print("Accuracy: " + str(accuracy))

    ## Correlation between models
    print("Correlation")
    predictions = []

    for m in tqdm(models):
        predictions.append(np.argmax(m.predict(x_test), axis=1))
    classified = []

    for prediction in tqdm(predictions):
        classified.append([1 if i==j else 0 for i,j in zip(prediction,y_test)])
    correlation_matrix = []

    for ix, x in enumerate(classified):
      row = []
      
      for iy, y in enumerate(classified):
        if (ix == iy):
          row.append(np.nan)
        else:
          row.append(pearsonr(x,y)[0])

      correlation_matrix.append(row)

    correlation_matrix = np.array(correlation_matrix)
    correlation_matrix_df = pd.DataFrame(correlation_matrix)
    correlation_matrix_df.to_csv(PATH + MODEL_NAME + f"/{run}/correlation_matrix.csv")
    
    display(correlation_matrix_df)
    correlation = np.nanmean(correlation_matrix.flatten())
    print("Average correlation: " + str(correlation))

    # Save the results
    file = PATH + MODEL_NAME + "/results.csv"
    df = pd.DataFrame([[run, accuracy, correlation]])

    if not os.path.isfile(file):
      df.to_csv(file, header=["run", "accuracy", "correlation"], index=False)
    else: # else it exists so append without writing the header
      df.to_csv(file, mode='a', header=False, index=False)

    clear_output(wait=True)